<br/><font size=10>Data organization</font><br/>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span><ul class="toc-item"><li><span><a href="#Dataset-description" data-toc-modified-id="Dataset-description-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Dataset description</a></span></li><li><span><a href="#Load-dataset" data-toc-modified-id="Load-dataset-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Load dataset</a></span></li></ul></li><li><span><a href="#Resampling" data-toc-modified-id="Resampling-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Resampling</a></span><ul class="toc-item"><li><span><a href="#Resampling-regarding-sampling-rate" data-toc-modified-id="Resampling-regarding-sampling-rate-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Resampling regarding sampling rate</a></span></li><li><span><a href="#Resampling-regarding-data-size" data-toc-modified-id="Resampling-regarding-data-size-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Resampling regarding data size</a></span></li></ul></li><li><span><a href="#Filtering" data-toc-modified-id="Filtering-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Filtering</a></span></li><li><span><a href="#Reference" data-toc-modified-id="Reference-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Reference</a></span></li></ul></div>

# Introduction

In this tutorial, we use the data from EEG Motor Movement/Imagery Dataset (EEG-MMIDB)[<sup>1</sup>](#refer-anchor-1). We offer the cleaned dataset that extract from the EEG-MMIDB that contains 109 subjects. In this tutorial, for simplicity, we take the first subject as a subset to show how the code works. The ready-to-use dataset can be find in the *dataset* folder.


<!-- > Dataset_1: the data of the first subject in the EEG-MMIDB.  
> Dataset_2: data of all 109 subjects of the EEG-MMIDB.

* Note: in the following models of this BCI tutorial, we will use Dataset_1 to run examples for the computational advantage. -->

## Dataset description
原始資料集描述可以在這裡找到：https://archive.physionet.org/pn4/eegmmidb/

經過我們的清理和排序後，每個 .npy 檔案包含一個受試者的數據。每個 npy 檔案的數據形狀為 [N, 65]，前 64 columns 代表 64 個 EEG 通道的讀數，最後一 columns 表示 class/ intent 標籤。行表示信號收集中的時間點，每 rows 代表特定時間點的一個讀數。在本教程中，我們稱每 rows 為一個實例。EEG-MMIDB 的取樣率為 160 Hz，這意味著設備每秒可以生成 160 個實例/ rows /時間點。

N 對不同的受試者有所不同，但 N 應該是 259,520 或 255,680。這是原始資料集中的固有差異。回想一下，取樣率是 160 Hz，因此，一些試驗持續 4.1 秒，而其他試驗持續 4.2 秒：4.1 秒（656=4.1 $\times$ 160 實例）或 4.2 秒（672 = 4.2 $\times$ 160 實例）。建議將信號分段為每秒。

根據實驗設置，我們將所有 EEG 信號分為 11 種不同的 cognitive intentions 如下。其中，帶有 image 的意圖表示受試者僅想像動作而不實際移動：這四種 intentions（標記為 4、5、8 和 9）嚴格來說是 motion imagery EEG。其餘的 intention 則是用戶在進行特定動作時的心理狀態。

> 標籤：  （看最後一行）    
0: open eyes,  
1: close eyes.  
2: left hand,  
3: right hand.  
4: image left hand,  
5: image right hand.  
6: open fists,  
7: open feet.  
8: image fist,  
9: image feet.  
10: rest.

## Load dataset

Next, we show how to load the data in 2 commands!

In [2]:
import numpy as np
dataset_1=np.load('1.npy') 
print('The shape of Dataset_1:', dataset_1.shape)
dataset_1

The shape of Dataset_1: (259520, 65)


array([[-16, -29,   2, ..., -11,  15,   0],
       [-56, -54, -27, ...,   1,  21,   0],
       [-55, -55, -29, ...,  18,  35,   0],
       ...,
       [  0,   0,   0, ...,   0,   0,   9],
       [  0,   0,   0, ...,   0,   0,   9],
       [  0,   0,   0, ...,   0,   0,   9]], dtype=int64)

如上所見，受試者 1 的數據包含 259520 個 instances 和 64 個 channels，而最後一 columns 表示 class 標籤（ground truth）。

我們首先介紹一些關於數據組織的術語：
* Instance（time step 或 time point）。每個 instance 表示在單個時間點或取樣點收集的一組值。例如，對於取樣率為 160 Hz 的設備，在 1 秒內將有 160 個 instances。我們交替使用 instance 和 time point。

* Segment（sample）。每個 segment 包含多個連續的 instances，可以表示大腦信號的特定事件/狀態。segment 的長度稱為 time window。我們交替使用 segment 和 sample。

# Resampling


有時我們需要在進行任何分析之前調整 signal frequencies 或樣本比例。Resampling是一種從原始數據樣本中抽取重複樣本的方法<sup>2</sup>。有兩種類型的重取樣被廣泛使用：
* Downsampling
* Upsampling

接下來，我們介紹在不同情況下重取樣的實現。

## Resampling regarding sampling rate

在兩種情況下對輸入數據 Resampling 

* 第一種是處理multimodal signals。例如，我們想利用兩種不同的輸入信號：sampling rate 為 1000 Hz 的腦信號（EEG）和取樣率為 20 Hz 的心跳信號（ECG）。為了統一輸入數據並將它們整合在一起，我們需要將它們調整到一個固定的取樣率，比如說 200 Hz。實際操作很簡單，即從 EEG 的每五個連續 instances 中取一個 instance（downsampling；同時，將 ECG 中的每個 instance 重複 10 次（upsampling）

* Resampling 也適用於輸入信號具有非常高的sampling rate而我們不需要的情況。以取樣率為 1000 Hz 的 EEG 數據為例：一個背景知識是 EEG 中最有用的信息在 70 Hz 以下，根據 Nyquist Sampling Theorem，我們只需要取樣率約為 140 Hz 的信號。為了減少計算成本，將原始高頻信號（1000 Hz）resample為低頻信號（例如 160 Hz）不是必須的，但可以接受。

## Resampling regarding data size

用以保持數據集（包括training and testing set）之間的平衡。例如，我們有 1000 個 positive samples 但只有 200 個 negative samples，這在機器學習中稱為 *imbalance*。分類器可以通過盲目預測最頻繁的類別（即將所有預測都設為 positive prediction）來達到 80% 的準確率。為了解決這個問題，有三種典型的解決方案：
1. Downsampling 最頻繁的類別直到達到更平衡的分佈（例如，隨機選擇多數類別的樣本並從訓練數據集中刪除它們）。
2. Upsampling 最不頻繁的類別（例如，隨機複製少數類別的樣本）。
3. 使用其他評估指標，如 ROC 曲線和 AUC（ROC 曲線下面積），這可以緩解數據不平衡問題。

*在本教程中，我們不需要 Resampling，因為我們的數據具有適當的取樣頻率並且已經平衡。*

# Filtering

Sometimes we need to filter the EEG signals into frequency bands of interests, in order to find the most informative and distinguishable patterns. 

從任何典型的 EEG 硬體收集的 EEG 信號基於與特定行為狀態的 strong intraband correlation ，具有 nonoverlapping frequency bands（Delta、Theta、Alpha、Beta 和 Gamma）。每個 EEG 模式包含與特定大腦信息相關的信號。意識程度表示個體在面對外部刺激時的感知。這主要在生理學而非心理學中定義。<sup>3</sup>

每個 frequency band 代表一種大腦狀態和對意識質性的評估：
* Delta pattern (0.5--4 Hz) 對應於受試者意識較低時的深度睡眠。
- Theta pattern (4--8 Hz) 對應於低意識範疇內的淺睡眠。
* Alpha pattern (8--12 Hz) 主要出現在閉眼和深度放鬆狀態，對應於中等意識。
* Beta pattern (12--30 Hz) 是受試者睜眼時的主要節律，與高意識相關。Beta 模式捕捉我們大多數的日常活動（如吃飯、走路和說話）。
* Gamma pattern (30--100 Hz) 代表多個大腦區域共同進行特定運動和認知功能的互動。

接下來，我們提供 filtering codes，以 Delta band 為例（本教程中不需要過濾，只是提供代碼以防讀者需要）。這裡使用的過濾器是 3-order band-pass butterworth filter。請調整 sampling frequency (fs)、lowcut 和 highcut 來定制自己的過濾器。

In [ ]:
# def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
#     b, a = butter_bandpass(lowcut, highcut, fs, order=order) # butter_bandpass 是可以把 data 通過 lowcut 和 highcut 的訊號濾掉
#     y = scipy.signal.lfilter(b, a, data)
#     return y 
# def butter_bandpass(lowcut, highcut, fs, order=5):
#     nyq = 0.5 * fs
#     low = lowcut / nyq
#     high = highcut / nyq
#     b, a = butter(order, [low, high], btype='band')
#     return b, a

In [3]:
import myimporter  # 透過myimporter.py檔案，我們可以直接import ipynb檔案（要不然預設只能import py檔案）
from BCI_functions import *  # BCI_functions.ipynb contains some functions we might use multiple times in this tutorial 

n_fea = 64  # 64 channels
label = dataset_1[:, n_fea: n_fea+1]  # seperate label from feature
feature = dataset_1[:, 0:n_fea] 
feature_f=[]  # feature after filtering

# EEG Delta pattern decomposition
for i in range(feature.shape[1]):
    x = feature[:, i]
    fs = 160.0
    lowcut = 0.5
    highcut = 4.0
    y = butter_bandpass_filter(x, lowcut, highcut, fs, order=3)
    feature_f.append(y) 
    
feature_f=np.array(feature_f).T
print('The shape of filtered feature:',feature_f.shape)

data_f=np.hstack((feature_f,label))  # stack label to filtered feature 
print("The shape of dataset_1 after filtering:",data_f.shape)

importing Jupyter notebook from BCI_functions.ipynb
The shape of filtered feature: (259520, 64)
The shape of dataset_1 after filtering: (259520, 65)


# Reference

<div id="refer-anchor-1"></div>

- [1]  [Goldberger AL, Amaral LAN, Glass L, Hausdorff JM, Ivanov PCh, Mark RG, Mietus JE, Moody GB, Peng C-K, Stanley HE. PhysioBank, PhysioToolkit, and PhysioNet: Components of a New Research Resource for Complex Physiologic Signals. Circulation 101(23):e215-e220](http://circ.ahajournals.org/cgi/content/full/101/23/e215)

<div id="refer-anchor-2"></div>

- [2]  [Resampling Imbalanced Data and Applying Machine Learning Techniques](https://medium.com/better-programming/resampling-imbalanced-data-and-applying-ml-techniques-91ebce40ff4d)

<div id="refer-anchor-2"></div>

- [3]  [Zhang X, Yao L, Wang X, et al. A survey on deep learning-based non-invasive brain signals: recent advances and new frontiers[J]. Journal of Neural Engineering, 2020.](https://arxiv.org/abs/1905.04149)

